# Detection of Spatial and Functional Signatures for Environmental Measurements

This Jupyter Notebook serves as the computational framework accompanying the paper, **"Urban Contexts: A Geospatial Approach to Identifying In-Situ Measurement Sites for Urban Acoustic Environments"**. It provides reproducible workflows designed to analyze urban spatial and functional characteristics comprehensively, enabling researchers and planners to identify contextually significant urban areas for environmental and acoustic measurements. The methodologies integrate geospatial and statistical analyses, emphasizing the connection between urban form, function, and environmental variability.

## Objectives

1. **Develop Robust Methodologies for Urban Spatial Analysis**  
   Establish a detailed framework for evaluating urban form through morphometric indicators such as dimension, shape, intensity, and connectivity. These metrics form the foundation for understanding spatial patterns and variability within urban environments.

2. **Create Scalable Workflows for Extracting Urban Form and Function Metrics**  
   Utilize scalable and reproducible geospatial workflows to integrate functional data, including Points of Interest (POIs), land use, and demographic information. These workflows enable the analysis of urban activity and accessibility in diverse contexts.

3. **Develop Signature Types Based on Morphometric Variables**  
   Employ advanced clustering techniques, such as Gaussian Mixture Models (GMM), to classify urban areas into distinct signature types. These classifications capture the interplay between form and function, providing actionable insights for urban analysis and environmental measurement.

This notebook aims to provide a structured approach to urban research, offering tools for analyzing spatial and functional dynamics while addressing the challenges of variability and complexity in urban landscapes.

Acknowledgments

This analysis makes use of the **Momepy** Python library for urban morphology analysis. For detailed documentation and further insights, visit [Momepy Documentation](https://docs.momepy.org/en/stable/).



---

## Table of Contents

1. [Analysis of Form](#1-analysis-of-form)
   1. [Data Retrieval](#11-data-retrieval)
      1. [Building Footprints](#111-building-footprints)
      2. [Street Network](#112-street-network)
      3. [Spatial Barriers](#113-spatial-barriers)
   2. [Data Pre-Processing](#12-data-pre-processing)
      1. [Footprints Checks and Cleaning](#121-footprints-checks-and-cleaning)
      2. [Barriers Checks and Cleaning](#122-barriers-checks-and-cleaning)
   3. [Generation of Geographies](#13-generation-of-geographies)
      1. [Enclosures](#131-enclosures)
      2. [Enclosed Tessellation](#132-enclosed-tessellation)
   4. [Morphometric Analysis](#14-morphometric-analysis)
      1. [Primary Morphometric Characters](#141-primary-morphometric-characters)
      2. [Contextualization](#142-contextualization)
2. [Analysis of Function](#2-analysis-of-function)
   1. [Land Use and Population Density](#21-land-use-and-population-density)
   2. [Entertainment](#22-entertainment)
   3. [Culture](#23-culture)
   4. [Education](#24-education)
   5. [Health](#25-health)
   6. [Transportation](#26-transportation)
   7. [Green & Blue Spaces](#27-green--blue-spaces)
   8. [Trees](#28-trees)
   9. [NDVI](#29-ndvi)
3. [Cluster Analysis](#3-cluster-analysis)
   1. [Form](#31-form)
   2. [Function](#32-function)
   3. [Spatial Signatures](#33-spatial-signatures)
4. [Figures](#4-figures)
5. [Cluster Summary](#5-cluster-summary)

---


## 1. Analysis of Form
### 1.1 Data Retrieval

In [ ]:
#!pip install ipyparallel
import sys
sys.path.append("/your system path/")
import momepy_utils
import tobler
import warnings
import geopandas as gdp
import libpysal
import momepy
import osmnx as ox
import geopandas as gpd
import pandas as pd
import numpy as np
from tqdm import tqdm
import osmnx as ox
from pandana.loaders import osm
from shapely.geometry import Point
from clustergram import Clustergram
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt
from bokeh.io import output_notebook
from bokeh.plotting import show
import os
import ipyparallel as ipp


output_notebook()

#### 1.1.1 Building Footprints
Description and Python code related to building footprints.


In [ ]:
#specify a folder for storage
folder = "/your_storage_hd/"

# Suppress warnings
warnings.filterwarnings("ignore")

In [ ]:
#specify your city
#We can use ``OSMnx`` to quickly download data from OpenStreetMap. If you intend to download larger areas, we recommend using ``pyrosm`` instead.
place = 'Würzburg'
local_crs = 32634

In [ ]:
buildings = osmnx.geometries.geometries_from_place(place, tags={'building':True})
buildings.head()

In [ ]:
# Get the bounding box of the buildings
bbox = buildings.total_bounds
print('Bounding box:', bbox)

In [ ]:
#cleaning geometries
buildings.geom_type.value_counts()

In [ ]:
#re-project the data from WGS84 to the local projection in meters
buildings = buildings[buildings.geom_type == "Polygon"].reset_index(drop=True)
buildings = buildings[["geometry"]].to_crs(local_crs)

#### 1.1.2 Street Network
Description and Python code related to the street network.


In [ ]:
#build the street network
osm_graph = osmnx.graph_from_place(place, network_type='drive')
osm_graph = osmnx.projection.project_graph(osm_graph, to_crs=local_crs)
roads = osmnx.graph_to_gdfs(
    osm_graph, 
    nodes=False, 
    edges=True,
    node_geometry=False, 
    fill_edge_geometry=True
)
roads.head()

In [ ]:
roads = momepy.remove_false_nodes(roads)
roads = roads[["geometry"]]
roads["nID"] = range(len(roads))

#### 1.1.3 Spatial Barriers
Description and Python code related to spatial barriers.


In [ ]:
#upload aoi
admin = gpd.read_file(folder + "/UA_UrbanCore.gpkg")

In [ ]:
bounds = admin.to_crs(4326).total_bounds
bounds

In [ ]:
#Using `OSMnx` we can specify OSM tags, selecting which geometries should be downloaded. First we get water-related barriers.
tags = {'natural': ['water', 'coastline']}

water = ox.geometries_from_bbox(bounds[3], bounds[1], bounds[2], bounds[0], tags)
water = water.to_crs(streets.crs)
water[['natural', 'geometry']].to_parquet(folder + "/parquets/water.pq")

In [ ]:
#Then we get railway and filter out proper geometry type (we want LineStrings representing railway tracks) and remove tunnels and trams (those are not spatial barriers).
tags = {'railway': True}
railway = ox.geometries_from_bbox(bounds[3], bounds[1], bounds[2], bounds[0], tags)
railway = railway.to_crs(streets.crs)
railway = railway[railway.geom_type == 'LineString']
railway = railway[railway.tunnel != 'yes']
railway = railway[~railway.railway.isin(['miniature', 'tram'])]
railway[['railway', 'geometry']].to_parquet(folder + "/parquets/railway.pq")

### 1.2 Data Pre-Processing
#### 1.2.1 Footprints Checks and Cleaning
Description and Python code for checking and cleaning building footprints.


In [ ]:
# We can then use `momepy.CheckTessellationInput()` class to check for potential issues which may arise during enclosed tessellation.
check = momepy.CheckTessellationInput(buildings)

In [ ]:
check = momepy.CheckTessellationInput(buildings)
#We see that some buildings would collapse (disappear) and some would be split, which may result in multipolygon tessellation cells.
check.collapse.area.max()
#If the maximum area of collapsed feature is low. we can safely remove them.

In [ ]:
buildings.shape

In [ ]:
buildings["uID"] = range(len(buildings))
buildings.to_parquet(folder + "parquets/buildings.pq")

#### 1.2.2 Barriers Checks and Cleaning
Description and Python code for checking and cleaning spatial barriers.

In [ ]:
admin.geom_type.unique()

In [ ]:
railway.geom_type.unique()

In [ ]:
#Roads seems to be fine, similarly to railway. The only step we do is extension of railway lines to snap to roads.
extended_railway = momepy.extend_lines(railway, 30, target=streets, extension=.1)

### 1.3 Generation of Geographies

#### 1.3.1 Enclosures
Enclosures require spatial barriers, which are roads and railway (we use the extended one we did above), limited by administrative boundary.

In [ ]:
%%time
enclosures = utils.momepy.enclosures(roads, limit=admin.iloc[[0]], additional_barriers=[extended_railway])

#### 1.3.2 Enclosed Tessellation
Description and Python code for generating enclosed tessellation.

In [ ]:
buildings = gpd.read_parquet(folder + "parquets/buildings.pq")

In [ ]:
%%time
tess = utils.momepy.Tessellation(buildings, 'uID', enclosures=enclosures)

In [ ]:
tess.tessellation.to_parquet(folder + "tessellation.pq")

In [ ]:
enclosures.to_parquet(folder + "enclosures.pq")

### 1.4 Morphometric Analysis

#### 1.4.1 Primary Morphometric Characters
Morphometric analysis begins with measuring primary characteristics, which are subsequently contextualized to reflect the tendencies and relationships within the local context of each tessellation cell.


In [ ]:
blg = gpd.read_parquet(folder + "buildings.pq")
streets = roads
tess = gpd.read_parquet(folder + "tessellation.pq")
enclosures = gpd.read_parquet(folder + "enclosures.pq")

In [ ]:
tess = tess.rename_geometry("tessellation").merge(
    blg[["uID", "geometry"]].rename_geometry("buildings"), on="uID", how="left"
)

In [ ]:
tess['tID'] = range(len(tess))

In [ ]:
tess

In [ ]:
blg = tess.set_geometry('buildings').dropna()

At this stage, we proceed to measure the morphometric characters. For detailed information on each metric, refer to the Momepy documentation. The results for each character are appended as new columns in the dataset. https://docs.momepy.org/

In [ ]:
%time blg['sdbAre'] = momepy.Area(blg).series
%time blg['sdbPer'] = momepy.Perimeter(blg).series
%time blg['sdbCoA'] = momepy.CourtyardArea(blg, 'sdbAre').series

%time blg['ssbCCo'] = momepy.CircularCompactness(blg, 'sdbAre').series
%time blg['ssbCor'] = momepy.Corners(blg).series
%time blg['ssbSqu'] = momepy.Squareness(blg).series
%time blg['ssbERI'] = momepy.EquivalentRectangularIndex(blg, 'sdbAre', 'sdbPer').series
%time blg['ssbElo'] = momepy.Elongation(blg).series

In [ ]:
%time cencon = momepy.CentroidCorners(blg)
blg['ssbCCM'] = cencon.mean
blg['ssbCCD'] = cencon.std

%time blg['stbOri'] = momepy.Orientation(blg).series
 
%time tess['stcOri'] = momepy.Orientation(tess).series

Next, we need to incorporate building orientation into the full dataframe alongside the tessellation data. To achieve this, we merge the datasets back together.

In [ ]:
tess = tess.merge(blg[['tID', 'stbOri']], on='tID', how='left')

In [ ]:
%time tess['stbCeA'] = (tess['stbOri'] - tess['stcOri']).abs()

In [ ]:
%time tess['sdcLAL'] = momepy.LongestAxisLength(tess).series
%time tess['sdcAre'] = momepy.Area(tess).series
%time tess['sscCCo'] = momepy.CircularCompactness(tess, 'sdcAre').series
%time tess['sscERI'] = momepy.EquivalentRectangularIndex(tess, 'sdcAre').series

%time tess['sicCAR'] = tess.buildings.area / tess['sdcAre']

In [ ]:
%time blg["mtbSWR"] = momepy.SharedWallsRatio(blg).series

Certain morphometric characters require spatial weights matrices for their computation. We can generate a Queen contiguity matrix based on the enclosed tessellation to capture the spatial relationships between tessellation cells.

In [ ]:
%time queen_1 = libpysal.weights.contiguity.Queen.from_dataframe(tess, ids="tID", geom_col='tessellation')

In [ ]:
%time tess["mtbAli"] = momepy.Alignment(tess.set_geometry("buildings"), queen_1, "tID", "stbOri").series

In [ ]:
%time tess["mtbNDi"] = utils.momepy.NeighborDistance(tess.set_geometry("buildings"), queen_1, "tID").series

In [ ]:
%time tess["mtcWNe"] = momepy.Neighbors(tess, queen_1, "tID", weighted=True).series
%time tess["mdcAre"] = momepy.CoveredArea(tess, queen_1, "tID").series

In certain scenarios, capturing the contiguity of buildings is essential. To achieve this, we can generate a Queen contiguity matrix based on the building layer, which considers shared edges and vertices between adjacent buildings.

In [ ]:
%time blg_q1 = libpysal.weights.contiguity.Queen.from_dataframe(blg, geom_col='buildings', silence_warnings=True)
 
%time blg["ldbPWL"] = momepy.PerimeterWall(blg, blg_q1).series

In [ ]:
%time blg["libNCo"] = utils.momepy.Courtyards(blg, spatial_weights=blg_q1).series

Morphometric characters often necessitate defining an immediate context for analysis. In our methodology, we utilize inclusive third-order contiguity, which extends the analysis to include not only directly adjacent units but also those within three levels of contiguity. This can be efficiently generated using the initial Queen contiguity matrix.

In [ ]:
%time queen_3 = momepy.sw_high(k=3, weights=queen_1)

In [ ]:
%time tess['ltbIBD'] = utils.momepy.MeanInterbuildingDistance(tess.set_geometry('buildings'), queen_1, 'tID', queen_3).series

To associate tessellation cells with streets, we perform an intersection operation. This process ensures that each tessellation cell is linked to the corresponding street segments it intersects. In cases where a tessellation cell intersects multiple street segments, we calculate the intersection ratio to proportionally distribute the relationship across these segments. This approach maintains spatial consistency and ensures accurate assignment.

In [ ]:
%time links = momepy.get_network_ratio(tess, streets)

In [ ]:
tess[['edgeID_keys', 'edgeID_values']] = links
#From these ratios we can get the primary link (the one which intersects the most).

In [ ]:
keys = tess.edgeID_values.apply(lambda a: np.argmax(a))
tess['edgeID_primary'] = [inds[i] for inds, i in zip(tess.edgeID_keys, keys)]

To ensure that the links between tessellation cells and street segments are available for both tessellation and building layers, we merge the results into the building dataset. This step integrates the spatial relationships into a unified framework, enabling consistent analysis across both layers.

In [ ]:
blg = blg.merge(tess[['tID', 'edgeID_primary']], on='tID', how='left')

To ensure that the initial integer index, used as edgeID, is preserved as an attribute of the streets, we add it explicitly to the street dataset. This precaution ensures that the edgeID remains intact, even if the data is shuffled or reordered later in the analysis. By preserving the edgeID as a dedicated column, we maintain a consistent reference for all subsequent operations and analyses.

In [ ]:
streets['edgeID_primary'] = range(len(streets))

In [ ]:
#Finally we are able to measure the characters combining multiple data sources.
%time streets["sdsLen"] = momepy.Perimeter(streets).series
%time tess["stcSAl"] = momepy.StreetAlignment(tess, streets, "stcOri", "edgeID_primary").series
%time blg["stbSAl"] = momepy.StreetAlignment(blg, streets, "stbOri", "edgeID_primary").series

In [ ]:
%time profile = momepy.StreetProfile(streets, blg, distance=3)
streets["sdsSPW"] = profile.w
streets["sdsSPO"] = profile.o
streets["sdsSWD"] = profile.wd

In [ ]:
%time streets["sssLin"] = momepy.Linearity(streets).series

In [ ]:
%%time 
# Area Covered by each edge

vals = {x:[] for x in range(len(streets))}
for i, keys in enumerate(tess.edgeID_keys):
    for k in keys:
        vals[k].append(i)
area_sums = []
for inds in vals.values():
    area_sums.append(tess.sdcAre.iloc[inds].sum())
streets['sdsAre'] = area_sums

In [ ]:
%%time
# Buildings per meter

bpm = []
for inds, l in zip(vals.values(), streets.sdsLen):
    bpm.append(tess.buildings.iloc[inds].notna().sum() / l if len(inds) > 0 else 0)
streets['sisBpM'] = bpm

In [ ]:
str_q1 = libpysal.weights.contiguity.Queen.from_dataframe(
    streets, silence_warnings=True
)

streets["misRea"] = momepy.Reached(
    streets,
    tess,
    "edgeID_primary",
    "edgeID_primary",
    spatial_weights=str_q1,
    mode="count",
).series

streets["mdsAre"] = momepy.Reached(
    streets,
    tess,
    "edgeID_primary",
    "edgeID_primary",
    spatial_weights=str_q1,
    mode="sum",
).series

Next, we create a graph representation and compute connectivity characteristics. The following cell constructs a networkX.MultiGraph, calculates connectivity metrics, and outputs two GeoDataFrames—one for the original segments and another for the nodes—along with spatial weights.

In [ ]:
%time graph = momepy.gdf_to_nx(streets)
 
print("node degree")
graph = momepy.node_degree(graph)
 
print("subgraph")
graph = momepy.subgraph(
    graph,
    radius=5,
    meshedness=True,
    cds_length=False,
    mode="sum",
    degree="degree",
    length="mm_len",
    mean_node_degree=False,
    proportion={0: True, 3: True, 4: True},
    cyclomatic=False,
    edge_node_ratio=False,
    gamma=False,
    local_closeness=True,
    closeness_weight="mm_len",
)
print("cds length")
graph = momepy.cds_length(graph, radius=3, name="ldsCDL")
 
print("clustering")
graph = momepy.clustering(graph, name="xcnSCl")
 
print("mean_node_dist")
graph = momepy.mean_node_dist(graph, name="mtdMDi")
 
%time nodes, edges, sw = momepy.nx_to_gdf(graph, spatial_weights=True)

In [ ]:
%time edges_w3 = momepy.sw_high(k=3, gdf=edges)
%time edges["ldsMSL"] = momepy.SegmentsLength(edges, spatial_weights=edges_w3, mean=True).series
 
%time nodes_w5 = momepy.sw_high(k=5, weights=sw)
%time nodes["lddNDe"] = momepy.NodeDensity(nodes, edges, nodes_w5).series
nodes["linWID"] = momepy.NodeDensity(
    nodes, edges, nodes_w5, weighted=True, node_degree="degree"
).series

In [ ]:
enclosures["ldeAre"] = momepy.Area(enclosures).series
enclosures["ldePer"] = momepy.Perimeter(enclosures).series
enclosures["lseCCo"] = momepy.CircularCompactness(enclosures, "ldeAre").series
enclosures["lseERI"] = momepy.EquivalentRectangularIndex(enclosures, "ldeAre", "ldePer").series
enclosures["lseCWA"] = momepy.CompactnessWeightedAxis(enclosures, "ldeAre", "ldePer").series
enclosures["lteOri"] = momepy.Orientation(enclosures).series
 
blo_q1 = libpysal.weights.contiguity.Queen.from_dataframe(enclosures, ids="eID")
 
inp, res = enclosures.sindex.query_bulk(enclosures.geometry, predicate='intersects')
indices, counts = np.unique(inp, return_counts=True)
enclosures['neighbors'] = counts - 1
enclosures['lteWNB'] = enclosures['neighbors'] / enclosures['ldePer']

In [ ]:
# Measure weighted cells within enclosure
encl_counts = tess.groupby('eID').count()
merged = enclosures[['eID', 'ldeAre']].merge(encl_counts[['tessellation']], how='left', on='eID')
enclosures['lieWCe'] = merged['tessellation'] / merged['ldeAre']

In [ ]:
tess['ltcWRE'] = momepy.BlocksCount(tess, 'eID', queen_3, 'tID').series

To link the data currently associated with nodes to the tessellation, we need to determine the nearest network-based node ID for each tessellation cell.

In [ ]:
# get node id
%time links = momepy.get_network_ratio(tess, edges)
tess[['edgeID_keys2', 'edgeID_values2']] = links
%time tess['nodeID'] = momepy.get_node_id(tess, nodes, edges, node_id='nodeID', edge_keys='edgeID_keys2', edge_values='edgeID_values2')

In [ ]:
%%time
nodes["sddAre"] = momepy.Reached(
    nodes, tess, "nodeID", "nodeID", mode="sum", values="sdcAre"
).series

In [ ]:
# Now we link all Dataframnes together
tess.columns

In [ ]:
blg.columns

In [ ]:
edges.columns

In [ ]:
nodes.columns

In [ ]:
enclosures.columns

In [ ]:
data = tess.merge(
    blg[
        [
            "tID",
            "sdbAre",
            "sdbPer",
            "sdbCoA",
            "ssbCCo",
            "ssbCor",
            "ssbSqu",
            "ssbERI",
            "ssbElo",
            "ssbCCM",
            "ssbCCD",
            "mtbSWR",
            "ldbPWL",
            "stbSAl",
            "libNCo",
        ]
    ],
    on="tID",
    how="left",
)
data = data.merge(
    edges[
        [
            "sdsLen",
            "sdsSPW",
            "sdsSPO",
            "sdsSWD",
            "sssLin",
            "sdsAre",
            "sisBpM",
            "misRea",
            "mdsAre",
            "ldsMSL",
            "edgeID_primary",
        ]
    ],
    on="edgeID_primary",
    how="left",
)
data = data.merge(
    nodes[
        [
            "degree",
            "meshedness",
            "proportion_3",
            "proportion_4",
            "proportion_0",
            "local_closeness",
            "ldsCDL",
            "xcnSCl",
            "mtdMDi",
            "nodeID",
            "lddNDe",
            "linWID",
            "sddAre",
        ]
    ],
    on="nodeID",
    how="left",
)
data = data.merge(
    enclosures[
        [
            "eID",
            "ldeAre",
            "ldePer",
            "lseCCo",
            "lseERI",
            "lseCWA",
            "lteOri",
            "lteWNB",
            "lieWCe",
        ]
    ],
    on="eID",
    how="left",
)

In [ ]:
data.columns

In [ ]:
data.to_parquet(folder + "data.pq")

In [ ]:
tess.to_parquet(folder + "tess.pq")
blg.to_parquet(folder + "blg.pq")
nodes.to_parquet(folder + "nodes.pq")
edges.to_parquet(folder + "edges.pq")
enclosures.to_parquet(folder + "enclosures.pq")

#### 1.4.2 Contextualization
Cluster analysis must be efficient for large datasets, so it avoids spatial constraints, as such algorithms typically do not scale well. However, our focus is on understanding the tendencies of characters within specific areas. Most primary characters are inherently local, offering minimal or no contextual insight. To address this, we contextualize each primary character by calculating the first, second, and third quartiles of the value distribution within an inclusive 10th-order contiguity around each tessellation cell, weighted by the inverse distance between the centroids of the cells.


In [ ]:
data = gpd.read_parquet(folder + "data.pq")
data.columns

Next, we specify the columns that exclusively represent the characters, excluding any other attributes such as geometry or IDs.

In [ ]:
characters = [
    "stcOri",
    "stbOri",
    "stbCeA",
    "sdcLAL",
    "sdcAre",
    "sscCCo",
    "sscERI",
    "sicCAR",
    "mtbAli",
    "mtbNDi",
    "mtcWNe",
    "mdcAre",
    "ltbIBD",
    "stcSAl",
    "ltcWRE",
    "sdbAre",
    "sdbPer",
    "sdbCoA",
    "ssbCCo",
    "ssbCor",
    "ssbSqu",
    "ssbERI",
    "ssbElo",
    "ssbCCM",
    "ssbCCD",
    "mtbSWR",
    "ldbPWL",
    "stbSAl",
    "libNCo",
    "sdsLen",
    "sdsSPW",
    "sdsSPO",
    "sdsSWD",
    "sssLin",
    "sdsAre",
    "sisBpM",
    "misRea",
    "mdsAre",
    "ldsMSL",
    "degree",
    "meshedness",
    "proportion_3",
    "proportion_4",
    "proportion_0",
    "local_closeness",
    "ldsCDL",
    "xcnSCl",
    "mtdMDi",
    "lddNDe",
    "linWID",
    "sddAre",
    "ldeAre",
    "ldePer",
    "lseCCo",
    "lseERI",
    "lseCWA",
    "lteOri",
    "lteWNB",
    "lieWCe",
]

We prepare a GeoDataFrame (gdf) that includes the morphometric characters alongside tessellation centroids as the geometry. Additionally, we generate a spatial weights matrix (W) representing the inclusive 10th order of contiguity.

In [ ]:
gdf = gpd.GeoDataFrame(data[characters], geometry=data.tessellation.centroid)
%time W = momepy.sw_high(k=10, weights=libpysal.weights.Queen.from_dataframe(data, geom_col='tessellation'))

Next, we iterate through the GeoDataFrame to calculate the contextualization for each character, capturing the first, second, and third quartiles of the value distributions within the inclusive 10th order of contiguity.

In [ ]:
convolutions = {}
for c in characters:
    convolutions[c] = []

# measure convolutions
for i, geom in tqdm(gdf.geometry.iteritems(), total=data.shape[0]):
    neighbours = W.neighbors[i]
    vicinity = gdf.iloc[neighbours]
    distance = vicinity.distance(geom)
    distance_decay = 1 / distance
    
    for c in characters:
        values = vicinity[c].values
        sorter = np.argsort(values)
        values = values[sorter]
        nan_mask = np.isnan(values)
        if nan_mask.all():
            convolutions[c].append(np.array([np.nan] * 3))
        else:
            sample_weight = distance_decay.values[sorter][~nan_mask]
            weighted_quantiles = np.cumsum(sample_weight) - 0.5 * sample_weight
            weighted_quantiles /= np.sum(sample_weight)
            interpolate = np.interp([.25, .5, .75], weighted_quantiles, values[~nan_mask])
            convolutions[c].append(interpolate)

In [ ]:
#resulting in a dictionary which can be exploited
%time conv = pd.DataFrame(convolutions, index=data.index)
%time exploded = pd.concat([pd.DataFrame(conv[c].to_list(), columns=[c + '_q1', c + '_q2',c + '_q3']) for c in characters], axis=1)

In [ ]:
exploded.index = data.tID
exploded

In [ ]:
exploded.to_parquet(folder + "convolutions.pq")

## 2. Analysis of Function

### 2.1 Land Use and Population Density
Description and Python code for analyzing land use and population density.


In [ ]:
import geopandas as gpd
import os

def process_geospatial_data(aoi_path, land_use_path, output_crs=32634):
    """
    Generalized function to process geospatial data for an area of interest and land use.
    
    Parameters:
        aoi_path (str): File path to the AOI (Area of Interest) GeoPackage.
        land_use_path (str): File path to the land use GeoPackage.
        output_crs (int): EPSG code for the desired Coordinate Reference System. Default is 32634.
    
    Returns:
        gpd.GeoDataFrame: Processed land use GeoDataFrame clipped to AOI.
    """
    try:
        # Load data
        aoi_gdf = gpd.read_file(aoi_path)
        land_use_gdf = gpd.read_file(land_use_path)

        # Coordinate Reference System Transformation
        aoi_gdf = aoi_gdf.to_crs(epsg=output_crs)
        land_use_gdf = land_use_gdf.to_crs(epsg=output_crs)

        # Clip and fix geometries
        land_use_gdf = gpd.clip(land_use_gdf, aoi_gdf)
        land_use_gdf.geometry = land_use_gdf.geometry.buffer(0)

        return land_use_gdf

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

if __name__ == "__main__":
    # Example usage
    aoi_path = input("Enter the file path for the AOI GeoPackage: ").strip()
    land_use_path = input("Enter the file path for the land use GeoPackage: ").strip()
    
    # Call the function
    result = process_geospatial_data(aoi_path, land_use_path)
    
    if result is not None:
        print("Data processed successfully!")
        # Save the result if needed
        save_path = input("Enter the file path to save the clipped land use GeoDataFrame: ").strip()
        result.to_file(save_path, driver="GPKG")
    else:
        print("Processing failed.")


In [ ]:

# function from tobler to ease the process
def area_max(source_df, target_df, variables):
    """
    Join attributes from source based on the largest intersection. In case of a tie it picks the first one.   
    """    
    target_df = target_df.copy()
    target_ix, source_ix = source_df.sindex.query_bulk(target_df.geometry, predicate='intersects')
    areas = target_df.geometry.values[target_ix].intersection(source_df.geometry.values[source_ix]).area

    main = []
    for i in range(len(target_df)):
        mask = target_ix == i
        if np.any(mask):
            main.append(source_ix[mask][np.argmax(areas[mask])])
        else:
            main.append(np.nan)
    
    main = np.array(main)
    mask = ~np.isnan(main)
    if pd.api.types.is_list_like(variables):
        for v in variables:
            arr = np.empty(len(main), dtype=object)
            arr[:] = np.nan
            arr[mask] = source_df[v].values[main[mask].astype(int)]
            target_df[v] = arr
    else:
        arr = np.empty(len(main), dtype=object)
        arr[:] = np.nan
        arr[mask] = source_df[variables].values[main[mask].astype(int)]
        target_df[variables] = arr
        
    return target_df


In [ ]:
# Apply the function
func_data = area_max(land_use, func_data, ['class_2018','Pop2018'])


### 2.2 Points of Interest (POI)

This section focuses on the analysis of Points of Interest (POIs) to understand the functional characteristics of urban areas. POIs are categorized into specific types to provide insights into different urban functionalities.

---

#### 2.2.1 Culture

**Description**:  
Analyzing cultural POIs, such as theaters, libraries, and cinemas, to understand the distribution of cultural landmarks in urban areas.

---

#### 2.2.2 Education

**Description**:  
Analyzing educational institutions, including schools, universities, and kindergartens, to map and evaluate educational accessibility.

---

#### 2.2.3 Health

**Description**:  
Analyzing health services, such as hospitals, clinics, and pharmacies, to assess the spatial distribution of healthcare facilities.

---

#### 2.2.4 Transportation

**Description**:  
Analyzing transportation hubs, including bus stations and bicycle rentals, to evaluate accessibility and mobility in urban areas.


In [ ]:
# Define the output folder and create it if it doesn't exist
output_folder = "output_amenities/"
os.makedirs(output_folder, exist_ok=True)

# Define the city and categories
city = "Würzburg, Germany"  # Update as needed
categories = {
    'entertainment': ["bar", "cafe", "nightclub", "restaurant"],
    'culture': ["place_of_worship", "theatre", "library", "cinema"],
    'education': ["school", "university", "library", "college", "kindergarten", "language_school", "music_school", "driving_school", "research_institute", "public_bookcase"],
    'health': ["hospital", "clinic", "dentist", "doctors", "pharmacy", "nursing_home", "veterinary", "social_facility"],
    'transportation': ["bus_station", "ferry_terminal", "taxi", "car_rental", "bicycle_rental", "parking", "fuel"]
}

# Function to process amenities by category
def process_amenities(category, tags, city, output_folder):
    """
    Processes and saves amenities for a given category.
    
    Parameters:
        category (str): The category name (e.g., 'entertainment').
        tags (list): List of OSM tags for the category.
        city (str): The city/place for downloading data.
        output_folder (str): Path to save the output GeoPackage.
    """
    print(f"Processing category: {category}")
    amenities_gdf = gpd.GeoDataFrame()

    for tag in tags:
        try:
            gdf = ox.geometries_from_place(city, tags={"amenity": tag})
            amenities_gdf = pd.concat([amenities_gdf, gdf], ignore_index=True)
        except Exception as e:
            print(f"Error processing tag '{tag}' in category '{category}': {e}")
    
    # Filter points only
    amenities_gdf = amenities_gdf[amenities_gdf['geometry'].apply(lambda x: isinstance(x, Point))]
    amenities_gdf["latitude"] = amenities_gdf["geometry"].y
    amenities_gdf["longitude"] = amenities_gdf["geometry"].x
    
    # Save to GeoPackage
    output_path = os.path.join(output_folder, f"{category}.gpkg")
    amenities_gdf.to_file(output_path, driver='GPKG')
    print(f"Saved {category} amenities to {output_path}")

# Process each category separately
for category, tags in categories.items():
    process_amenities(category, tags, city, output_folder)


In [ ]:
# Spatial join to link amenities to tessellation cells
tessellation = gpd.read_file("tessellation_cells.gpkg")
amenities = gpd.read_file(f"{output_folder}entertainment.gpkg")

joined = gpd.sjoin(tessellation, amenities, how="left", predicate="intersects")

# Aggregating amenities per tessellation
result = joined.groupby('tessellation_id').size().reset_index(name='amenity_count')



#### 2.2.5 Green & Blue Spaces

**Description**:  
Analyzing urban green and blue spaces, including parks, lakes, and rivers, to assess ecological and recreational resources in the city.

---

#### 2.2.6 Trees

**Description**:  
Analyzing urban trees to evaluate canopy coverage and biodiversity in urban landscapes.

---

#### 2.2.7 NDVI & LST

**Description**:  
Analyzing the Normalized Difference Vegetation Index (NDVI) to assess vegetation health and density using satellite imagery.

#### 2.2.7 NOISE

#### 2.2.8 Air Quality

In [ ]:
# Constants
WALKING_SPEED_M_PER_MIN = 83.3333  # Average walking speed in meters per minute (5 km/h)
POINT_INTERVAL = 100  # Interval in meters for placing points along polygon boundaries

def generate_boundary_points(polygon, interval):
    """
    Generate points at specified intervals along the boundary of a polygon.
    
    Args:
        polygon (Polygon or MultiPolygon): The geometry to generate points for.
        interval (int): Distance interval for generating points.
        
    Returns:
        list: List of Point objects along the boundary.
    """
    points = []
    if isinstance(polygon, MultiPolygon):
        for part in polygon.geoms:
            points.extend(generate_boundary_points(part, interval))
    elif isinstance(polygon, Polygon):
        num_points = int(np.ceil(polygon.length / interval))
        points = [polygon.boundary.interpolate(i / num_points, normalized=True) for i in range(num_points)]
    return points

def download_and_process_amenity(category, tags, city, output_folder, interval=POINT_INTERVAL):
    """
    Download and process green/blue space amenities, generating boundary points.
    
    Args:
        category (str): Category name for the amenity (e.g., 'parks', 'water').
        tags (dict): Tags for querying amenities from OpenStreetMap.
        city (str): Name of the city to query.
        output_folder (str): Path to save processed files.
        interval (int): Interval in meters for placing boundary points.
        
    Returns:
        gpd.GeoDataFrame: GeoDataFrame containing generated points.
    """
    print(f"Processing {category} data for {city}...")
    gdf = ox.geometries_from_place(city, tags=tags)
    gdf = gdf[gdf.geometry.type.isin(['Polygon', 'MultiPolygon'])]
    
    # Generate boundary points
    boundary_points = gdf['geometry'].apply(lambda x: generate_boundary_points(x, interval)).explode()
    points_gdf = gpd.GeoDataFrame(geometry=boundary_points, crs=gdf.crs)
    
    # Save the results
    output_path = os.path.join(output_folder, f"{category}_boundary_points.gpkg")
    points_gdf.to_file(output_path, driver='GPKG')
    print(f"{category.capitalize()} data saved to {output_path}")
    
    return points_gdf

def calculate_accessibility(network, amenities_gdf, category, distance_600m=600, distance_15min=None):
    """
    Calculate accessibility for a given category of amenities.
    
    Args:
        network (pandana.Network): Pandana network object for accessibility analysis.
        amenities_gdf (gpd.GeoDataFrame): GeoDataFrame containing amenities.
        category (str): Category name for the amenities.
        distance_600m (int): Distance in meters for calculating 600m accessibility.
        distance_15min (float): Distance in meters for calculating 15-min accessibility.
        
    Returns:
        pd.DataFrame: DataFrame with accessibility metrics added.
    """
    print(f"Calculating accessibility for {category}...")
    
    if distance_15min is None:
        distance_15min = WALKING_SPEED_M_PER_MIN * 15  # Default 15-minute walking distance
    
    # Set POIs in the network
    network.set_pois(category=category, maxdist=distance_15min, maxitems=len(amenities_gdf),
                     x_col=amenities_gdf.geometry.x, y_col=amenities_gdf.geometry.y)
    
    # Calculate accessibility metrics
    accessibility = {}
    accessibility[f"{category}_accessibility_600m"] = network.nearest_pois(
        distance=distance_600m, category=category, num_pois=len(amenities_gdf)
    ).replace(distance_600m, np.nan).count(axis=1)
    
    accessibility[f"{category}_accessibility_15min"] = network.nearest_pois(
        distance=distance_15min, category=category, num_pois=len(amenities_gdf)
    ).replace(distance_15min, np.nan).count(axis=1)
    
    print(f"Accessibility for {category} calculated.")
    return pd.DataFrame(accessibility)

if __name__ == "__main__":
    # Example usage
    city = "Würzburg, Germany"
    output_folder = "processed_green_blue_spaces"
    os.makedirs(output_folder, exist_ok=True)
    
    # Define categories and tags for green and blue spaces
    amenities = {
        "parks": {"leisure": "park"},
        "water": {"natural": ["water", "riverbank"], "waterway": "river"}
    }
    
    # Download and process amenities
    amenities_gdfs = {}
    for category, tags in amenities.items():
        amenities_gdfs[category] = download_and_process_amenity(category, tags, city, output_folder)
    
    # Initialize Pandana network (replace with your network setup)
    # Example: network = pandana.Network(nodes_x, nodes_y, edges)
    network = None  # Replace with actual network initialization
    
    # Calculate accessibility for each category
    for category, gdf in amenities_gdfs.items():
        if network is not None:
            accessibility_metrics = calculate_accessibility(network, gdf, category)
            print(accessibility_metrics.head())
        else:
            print("Network not initialized. Skipping accessibility calculations.")


In [ ]:
#NDVI
%%time 
stats = rasterstats.zonal_stats(
    func_data.geometry, 
    raster=folder + "ndvi/2017_NDVI.tif", 
    stats=['min', 'max', 'median', 'mean']
)

In [ ]:
statsdf = pd.DataFrame(stats, index=func_data.index)
statsdf

In [ ]:
func_data['ndvi_min'] = statsdf['min']
func_data['ndvi_max'] = statsdf['max']
func_data['ndvi_mean'] = statsdf['mean']
func_data['ndvi_median'] = statsdf['median']

In [ ]:
# Similar processing for lst levels
# Perform zonal stats for LST (Land Surface Temperature)
lst_stats = zonal_stats(data.geometry, raster=folder + "/sat/LST_S_2023-06-01_2023-09-01.tif",
                        stats=['max'])
lst_statsdf = pd.DataFrame(lst_stats)

# Rename columns in lst_statsdf to prevent overlap
lst_statsdf.columns = [f'lst_{col}' for col in lst_statsdf.columns]

# Join the dataframes
func_data = func_data.join(lst_statsdf)


In [ ]:

# Paths to the input files
tessellation_file = "path_to_tessellation.gpkg"  
noise_file = "path_to_noise_layer.tif"  # Noise raster layer (L_den > 55 dB(A))

# Load tessellation cells
tessellation_gdf = gpd.read_file(tessellation_file)

# Ensure the CRS of tessellation matches the noise raster
tessellation_gdf = tessellation_gdf.to_crs(epsg=32634)  # Update to match CRS of noise raster

# Perform zonal statistics to extract maximum noise levels
noise_stats = zonal_stats(
    tessellation_gdf.geometry,
    noise_file,
    stats=['max']
)

# Convert the results into a DataFrame and rename columns
noise_stats_df = pd.DataFrame(noise_stats)
noise_stats_df.columns = [f'noise_{col}' for col in noise_stats_df.columns]

# Join noise data with tessellation GeoDataFrame
tessellation_gdf = tessellation_gdf.join(noise_stats_df)

# Save the updated tessellation with noise data
output_file = "tessellation_with_noise.gpkg"
tessellation_gdf.to_file(output_file, driver="GPKG")


In [ ]:
import requests
import pandas as pd

# Fetching data from the sensor.community API
url = "https://data.sensor.community/airrohr/v1/filter/country=DE"
response = requests.get(url)
if response.status_code == 200:
    data = response.json()
else:
    print("Failed to fetch data: Status code", response.status_code)
    data = []

# Convert to DataFrame
sensor_data = pd.DataFrame(data)

# Filter for stations in Würzburg (approximate coordinates)
wurzburg_lat_range = (49.75, 49.85)
wurzburg_lon_range = (9.90, 10.00)

# Extracting location information and filtering
sensor_data['latitude'] = sensor_data['location'].apply(lambda x: float(x['latitude']))
sensor_data['longitude'] = sensor_data['location'].apply(lambda x: float(x['longitude']))
wurzburg_data = sensor_data[
    (sensor_data['latitude'].between(*wurzburg_lat_range)) & 
    (sensor_data['longitude'].between(*wurzburg_lon_range))
]

# Extract PM2.5 and PM10 measurements
def extract_pm_values(sensordatavalues):
    pm_values = {'PM2.5': None, 'PM10': None}
    for record in sensordatavalues:
        if record['value_type'] == 'P1':  # PM10
            pm_values['PM10'] = float(record['value'])
        elif record['value_type'] == 'P2':  # PM2.5
            pm_values['PM2.5'] = float(record['value'])
    return pm_values

wurzburg_data['PM_values'] = wurzburg_data['sensordatavalues'].apply(extract_pm_values)

# Displaying the filtered data
wurzburg_data[['timestamp', 'PM_values']].head()


In [ ]:
import geopandas as gpd
from shapely.geometry import Point

# Convert PM data to a GeoDataFrame
pm_data = pd.DataFrame(wurzburg_data['PM_values'].tolist())
pm_data['latitude'] = wurzburg_data['latitude']
pm_data['longitude'] = wurzburg_data['longitude']
pm_data['geometry'] = [Point(lon, lat) for lon, lat in zip(pm_data['longitude'], pm_data['latitude'])]
pm_geo_data = gpd.GeoDataFrame(pm_data, geometry='geometry')

# Set the CRS for pm_geo_data to match the land_use GeoDataFrame
# Replace 'EPSG_Code' with the correct EPSG code of your land_use data
pm_geo_data.set_crs(epsg=32634, inplace=True)


In [ ]:
# Perform the spatial join
pm_land_use = gpd.sjoin(land_use, pm_geo_data, how='left', op='intersects')

In [ ]:
# Aggregate PM values by land use class
# Replace 'land_use_class_column' with the actual column name for land use classes
pm_aggregated = pm_land_use.groupby('class_2018').agg({'PM2.5': 'mean', 'PM10': 'mean'}).reset_index()


In [ ]:
# Merge the aggregated PM data into combined_data
# Ensure the land use class column name matches in both dataframes
combined_data = combined_data.merge(pm_aggregated, on='class_2018', how='left')


In [ ]:
# load the data
stadtbezirke_hauptwohnsitz_altersgruppen = '...stadtbezirke_hauptwohnsitz_altersgruppen.csv'  # Update the filename if necessary
haushalte_df = gpd.read_file(stadtbezirke_hauptwohnsitz_altersgruppen)


stadtbezirke_haushalte_durchschnittsgroesse = 'stadtbezirke_haushalte_durchschnittsgroesse.csv'  # Update the filename if necessary
hauptwohnsitz_df = gpd.read_file(stadtbezirke_haushalte_durchschnittsgroesse)

In [ ]:
from shapely.geometry import Point
import geopandas as gpd

# Convert 'Geo-Punkt' to geometry
def extract_geometry(geo_punkt):
    if geo_punkt:
        lat, lon = map(float, geo_punkt.split(', '))
        return Point(lon, lat)
    return None

hauptwohnsitz_df['geometry'] = hauptwohnsitz_df['Geo-Punkt'].apply(extract_geometry)
hauptwohnsitz_gdf = gpd.GeoDataFrame(hauptwohnsitz_df, geometry='geometry')
hauptwohnsitz_gdf.set_crs(epsg=4326, inplace=True)  # Assuming WGS 84 coordinate system


In [ ]:
# Merge the two dataframes on 'Stadtbezirk'
merged_df = pd.merge(haushalte_df, hauptwohnsitz_gdf[['Stadtbezirk', 'geometry']], on='Stadtbezirk', how='left')


In [ ]:
econ_census = gpd.read_file("census_2011_wue.geojson")

In [ ]:
econ_census = econ_census.to_crs(nodes.crs)

In [ ]:
%%time
network.set_pois(category = 'pois',
                 maxdist = 1200,
                 maxitems=10000,
                 x_col = econ_census.geometry.x, 
                 y_col = econ_census.geometry.y)

In [ ]:
%%time
nodes['pois'] = network.nearest_pois(distance = 1200,
                               category = 'pois',
                               num_pois = 10000,
                               include_poi_ids = False).replace(1200, pd.NA).count(axis=1)

In [ ]:
nodes['pois'].max()

## 3. Cluster Analysis
Cluster analysis leverages data on urban form, function, and their combination to identify homogeneous patterns of built form and activity. We employ a Gaussian Mixture Model (GMM), which provides a probabilistic approach to clustering. This method allows for overlapping clusters, capturing the complexity and subtlety of urban environments more effectively.

### 3.1 Form
Description and Python code for clustering based on form.


In [ ]:
form = pd.read_parquet(folder + "convolutions.pq")
form

In [ ]:
#Raw data needs to be standardised. We use standard scaler.
scaler = preprocessing.StandardScaler()
data = scaler.fit_transform(form)

In [ ]:
np.isnan(data).sum()

In [ ]:
data[np.isnan(data)] = 0

In [ ]:
# Initialize and fit the Clustergram using GMM
cgram = Clustergram(range(1, 30), method='gmm', covariance_type='full', random_state=42)
cgram.fit(data)

In [ ]:
# Plot the resulting clustergram
ax = cgram.plot(figsize=(20, 10), linewidth=0.5, cluster_style={"edgecolor": "blue", "alpha": 0.6}, size=1,
                line_style={"alpha": 0.5})
plt.title("Clustergram Using Gaussian Mixture Model")
plt.xlabel("Number of Clusters")
plt.ylabel("Cluster Assignment Stability")
plt.show()

In [ ]:
# Determine the optimal number of clusters
optimal_clusters = cgram.optimal_k_
print(f"Optimal number of clusters based on GMM: {optimal_clusters}")


In [ ]:
# Assign clusters to the data based on the optimal number
gmm_labels = cgram.labels_[optimal_clusters]

In [ ]:
# Assign the cluster labels to tessellation data
tess = gpd.read_parquet(folder + "tess.pq")
tess['clusters_form'] = gmm_labels

In [ ]:
# Save the tessellation with clusters
tess.to_file(folder + "tess_with_gmm_clusters_form.gpkg", driver="GPKG")


In [ ]:
# Plot spatial distribution of clusters
ax = tess.plot('clusters_form', categorical=True, legend=True, figsize=(20, 20), cmap='tab20')
ax.set_axis_off()
plt.title(f"Spatial Clusters (GMM, {optimal_clusters} Clusters)")
plt.show()

In [ ]:
ax = tess.set_geometry('buildings').plot('clusters_form', categorical=True, legend=True, figsize=(20, 20), cmap='tab20')
ax.set_axis_off()

## Generate Spatial Signatures
As the final step, we create geometries representing spatial signatures by combining contiguous tessellation cells that belong to the same cluster. To efficiently dissolve the large number of polygons involved, we utilize dask-geopandas for parallel processing.

In [ ]:
client = Client(LocalCluster(n_workers=16))
client

In [ ]:
tess = gpd.read_parquet(folder + "tess.pq", columns=['tessellation']).rename_geometry("geometry")
clusters = pd.read_csv(folder + "FINAL_cluster_labels.csv", index_col=0)

dask_dissolve mimics the behaviour of geopandas.dissolve, just based on parallel implementation using dask-geopandas.

In [ ]:
%%time
ddf = dask_geopandas.from_geopandas(tess.sort_values('cluster'), npartitions=64)
final = dask_dissolve(ddf, by='cluster').compute()

In [ ]:
final.plot('cluster', categorical=True, figsize=(20, 20), cmap='tab20')

In [ ]:
## 4. Figures


In [ ]:
# Load data
final = gpd.read_parquet(folder + "signatures.pq")
enc = gpd.read_parquet(folder + "enclosures.pq")


In [ ]:
# Set up styling
sns.set(context="paper", style="ticks", rc={'patch.force_edgecolor': False})


In [ ]:
# Create colormap
cmap = ugg.get_colormap(final.cluster.nunique(), randomize=False)
gradient = np.linspace(0, 1, 256)
gradient = np.vstack((gradient, gradient))

In [ ]:
# Generate symbology dynamically
cols = cmap.colors
symbology = {i: cols[i % len(cols)] for i in range(final.cluster.nunique())}


In [ ]:
# Placeholder names for clusters
names = {i: f"Cluster {i}" for i in range(final.cluster.nunique())}

# Prepare data for plotting
df = final.set_crs(enc.crs).to_crs(3857)
token = ""

# Plot clusters
ax = df.plot(
    color=df['cluster'].map(symbology),
    figsize=(20, 20),
    zorder=1,
    linewidth=0.3,
    edgecolor='w',
    alpha=1
)

In [ ]:

# Add basemaps
contextily.add_basemap(ax, crs=df.crs, source=ugg.get_tiles('roads', token), zorder=2, alpha=0.3)
contextily.add_basemap(ax, crs=df.crs, source=ugg.get_tiles('labels', token), zorder=3, alpha=1)
contextily.add_basemap(ax, crs=df.crs, source=ugg.get_tiles('background', token), zorder=-1, alpha=1)


In [ ]:
# Add scalebar
scalebar = ScaleBar(
    dx=1,
    color=ugg.COLORS[0],
    location='lower right',
    height_fraction=0.002,
    pad=0.5,
    frameon=False,
)
ax.add_artist(scalebar)

In [ ]:
# Add north arrow
ugg.north_arrow(
    plt.gcf(),
    ax,
    0,
    size=0.026,
    linewidth=1,
    color=ugg.COLORS[0],
    loc="upper left",
    pad=0.002,
    alpha=0.9
)

In [ ]:
# Add legend
custom_points = [
    Line2D([0], [0], marker="o", linestyle="none", markersize=10, color=color) 
    for color in symbology.values()

In [ ]:
leg_points = ax.legend(
    custom_points,
    [f"{name}" for name in names.values()],
    loc='upper right',
    frameon=True
)
ax.add_artist(leg_points)

In [ ]:
# Save the final map
plt.savefig(folder + "signatures.png", dpi=300, bbox_inches="tight")

## 4. Contextual Purposive Sampling